# 基于朴素贝叶斯的垃圾邮件识别

使用朴素贝叶斯分类器，对垃圾邮件进行分类。基于贝叶斯定理，通过对邮件中的出现的词语评估邮件是垃圾邮件的概率，从而给出预测结果。

如，判断单词Free出现的情况下，是垃圾邮件的概率：

$$
P(Spam|Free)=\frac{P(Spam)P(Free|Spam)}{P(Free)}
$$

分类器应该检查邮件中的每个单词，并确定每个单词对结果评估有多大贡献，所以我们需要用所有可能的单词去训练模型，但要除去"a"、"the"、"and"这种无意义的词。当检查了邮件中所有的单词，可以把每个单词所确定的垃圾邮件的概率相乘，从而得到最终的概率。

In [1]:
import os
import io
import numpy
from pandas import DataFrame
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

def readFiles(path):
    for root, dirnames, filenames in os.walk(path):
        for filename in filenames:
            path = os.path.join(root, filename)

            inBody = False
            lines = []
            with io.open(path, 'r', encoding='latin1') as f:
                for line in f:
                    if inBody:
                        lines.append(line)
                    elif line == '\n':
                        inBody = True
            message = '\n'.join(lines)
            yield path, message


def dataFrameFromDirectory(path, classification):
    rows = []
    index = []
    for filename, message in readFiles(path):
        rows.append({'message': message, 'class': classification})
        index.append(filename)
    return DataFrame(rows, index=index)


data = DataFrame({'message': [], 'class': []})

import pandas as pd
data = pd.concat([data, dataFrameFromDirectory('./data/emails/spam', 'spam')])
data = pd.concat([data, dataFrameFromDirectory('./data/emails/ham', 'ham')])


In [2]:
data.head()

,message,class
./data/emails/spam\00001.7848dde101aa985090474a91ec93fcf0,"<!DOCTYPE HTML PUBLIC ""-//W3C//DTD HTML 4.0 Tr...",spam
./data/emails/spam\00002.d94f1b97e48ed3b553b3508d116e6a09,1) Fight The Risk of Cancer!\n\nhttp://www.adc...,spam
./data/emails/spam\00003.2ee33bc6eacdb11f38d052c44819ba6c,1) Fight The Risk of Cancer!\n\nhttp://www.adc...,spam
./data/emails/spam\00004.eac8de8d759b7e74154f142194282724,##############################################...,spam
./data/emails/spam\00005.57696a39d7d84318ce497886896bf90d,I thought you might like these:\n\n1) Slim Dow...,spam
